In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import sqlite3
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib.pyplot as plt
from uuid import uuid4

## Menggabungkan Dataset

In [ ]:
## Pembuatan Customer ID (data dummy) ##
a = np.random.randint(10, high=100, size=1000)
b = np.random.randint(100, high=1000, size=1000)
c = np.random.randint(100, high=1000, size=1000)
d = np.random.randint(100, high=1000, size=1000)

CustomerID = []
for i in range(1000):
    CustomerID.append(f'{a[i]}-{b[i]}-{c[i]}-{d[i]}')

Duplicate = False     # Memastikan tidak ada entri duplikat
for i in range(1000):
    if CustomerID.count(CustomerID[i]) > 1:
        Duplicate = True

In [ ]:
## Pembuatan Product ID (data dummy) ##
a = np.random.randint(1000, high=10000, size=1000)
b = np.random.randint(100, high=1000, size=1000)
c = np.random.randint(100, high=1000, size=1000)

ProductID = []
for i in range(1000):
    ProductID.append(f'{a[i]}-{b[i]}-{c[i]}')

Duplicate = False     # Memastikan tidak ada entri duplikat
for i in range(1000):
    if ProductID.count(ProductID[i]) > 1:
        Duplicate = True

In [ ]:
# Import Database
df_customer = pd.read_csv("customer.csv", sep=';')
df_sales = pd.read_csv("sales.csv")
df_sales.drop(columns=["Gender"], inplace=True)

In [ ]:
# Gabungkan data customer kedalam data sales
for column in df_customer.columns:
    df_sales.insert(0,column,df_customer[column])
    
# Tambah CustomerID dan ProductID
df_sales.insert(0, "CustomerID", CustomerID)
df_sales.insert(0, "ProductID", ProductID)

In [ ]:
# Export Data keseluruhan menjadi file CSV
df_sales.to_csv('Database_data.csv')

##Slicing Dataset

In [ ]:
df = pd.read_csv('Database_data.csv')
df.head()

,Unnamed: 0,ProductID,CustomerID,Phone Number,Address,gender,Name,Invoice ID,Branch,City,...,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,0,2501-265-751,43-107-331-695,81573975928,777 Brockton Avenue,m,Hafizhan Shidqi,750-67-8428,A,Yangon,...,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,1,1039-533-780,75-364-901-710,81573918390,30 Memorial Drive,m,Gandhi Wibowo,226-31-3081,C,Naypyitaw,...,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,2,5992-444-146,93-888-168-812,81573860852,250 Hartford Avenue,m,Aldio Mahendra Purwandrarto,631-41-3108,A,Yangon,...,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,3,5099-512-798,56-981-748-179,81573803314,700 Oak Street,m,Benny Putra,123-19-1176,A,Yangon,...,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,4,6148-845-651,99-154-337-194,81573745776,66-4 Parkhurst Rd,m,Vicky Vernando Dasta,373-73-7910,A,Yangon,...,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               1000 non-null   int64  
 1   ProductID                1000 non-null   object 
 2   CustomerID               1000 non-null   object 
 3   Phone Number             1000 non-null   int64  
 4   Address                  1000 non-null   object 
 5   gender                   1000 non-null   object 
 6   Name                     1000 non-null   object 
 7   Invoice ID               1000 non-null   object 
 8   Branch                   1000 non-null   object 
 9   City                     1000 non-null   object 
 10  Customer type            1000 non-null   object 
 11  Product line             1000 non-null   object 
 12  Unit price               1000 non-null   float64
 13  Quantity                 1000 non-null   int64  
 14  Tax 5%                   

In [ ]:
customer = df[['CustomerID','Customer type','Phone Number','Name','Address','gender']]
product = df[['ProductID','Product line','Unit price','Tax 5%','Branch']]
purchase = df[['Invoice ID','ProductID','CustomerID','Name','Address','Quantity','Total','Date','Time','Payment','cogs','gross margin percentage','gross income','Rating']]
store = df[['Branch','City']]

In [ ]:
customer.head()

,CustomerID,Customer type,Phone Number,Name,Address,gender
0,43-107-331-695,Member,81573975928,Hafizhan Shidqi,777 Brockton Avenue,m
1,75-364-901-710,Normal,81573918390,Gandhi Wibowo,30 Memorial Drive,m
2,93-888-168-812,Normal,81573860852,Aldio Mahendra Purwandrarto,250 Hartford Avenue,m
3,56-981-748-179,Member,81573803314,Benny Putra,700 Oak Street,m
4,99-154-337-194,Normal,81573745776,Vicky Vernando Dasta,66-4 Parkhurst Rd,m


In [ ]:
customer_array = np.asarray(customer) 
print(customer_array)

[['43-107-331-695' 'Member' 81573975928 'Hafizhan Shidqi'
  '777 Brockton Avenue' 'm']
 ['75-364-901-710' 'Normal' 81573918390 'Gandhi Wibowo'
  '30 Memorial Drive' 'm']
 ['93-888-168-812' 'Normal' 81573860852 'Aldio Mahendra Purwandrarto'
  '250 Hartford Avenue' 'm']
 ...
 ['90-552-464-712' 'Member' 81516610542 'Diky Handriansyah'
  '454 Highland Ave' 'm']
 ['75-641-975-364' 'Normal' 81516553004 'Giono' '1184 Fall River Avenue'
  'm']
 ['11-836-689-420' 'Member' 81512361845 'Suryadi' '420 Greengrass Road'
  'm']]


In [ ]:
product.head()

In [ ]:
product_array = np.asarray(product)
print(product_array)

In [ ]:
product_array.shape

In [ ]:
purchase.head()

In [ ]:
purchase_array = np.asarray(purchase)
print(purchase_array)

In [ ]:
store.head()

In [ ]:
store_array = np.asarray(store)
print(store_array)

In [ ]:
array = np.asarray(df)

In [ ]:
print(array)

In [ ]:
MyDB = sqlite3.connect('e-commerce db.db')

In [ ]:
c = MyDB.cursor()

##Table Customer

In [ ]:
c.execute('''CREATE TABLE customer(
  "Customer_ID"	VARCHAR(14) NOT NULL PRIMARY KEY UNIQUE,
  "Customer_type"	VARCHAR(6) NOT NULL,
  "Phone_Number"	INTEGER NOT NULL,
  "Name"	VARCHAR(28) NOT NULL,
  "Address"	VARCHAR(30) NOT NULL,
  "Gender"	VARCHAR(1) NOT NULL
)''')

In [ ]:
c.executemany("INSERT INTO customer(Customer_ID,Customer_type,Phone_Number,Name,Address,Gender) VALUES(?,?,?,?,?,?)",customer_array)

##Table Product

In [ ]:
c.execute('''CREATE TABLE product(
  "Product_ID"	VARCHAR(12) NOT NULL PRIMARY KEY UNIQUE,
  "Product_line"	VARCHAR(22) NOT NULL,
  "Unit_price"	NUMERIC(5, 2) NOT NULL,
  "Tax_5"	NUMERIC(7, 4) NOT NULL,
  "Branch"	VARCHAR(1) NOT NULL,
  FOREIGN KEY (Branch) REFERENCES store(Branch))''')

In [ ]:
c.executemany("INSERT INTO product(Product_ID,Product_line,Unit_price,Tax_5,Branch) VALUES(?,?,?,?,?)",product_array)

##Table Purchase

In [ ]:
c.execute('''CREATE TABLE purchase(
  "Invoice_ID" VARCHAR(11) NOT NULL PRIMARY KEY UNIQUE,
  "Product_ID" VARCHAR(12) NOT NULL UNIQUE,
  "Customer_ID" VARCHAR(14) NOT NULL UNIQUE,
  "Name" VARCHAR(28) NOT NULL,
  "Address" VARCHAR(30) NOT NULL,
  "Quantity" INTEGER NOT NULL,
  "Total" NUMERIC(8, 4) NOT NULL,
  "Date" DATE NOT NULL,
  "Time" VARCHAR(5) NOT NULL,
  "Payment" VARCHAR(11) NOT NULL,
  "cogs" NUMERIC(6, 2) NOT NULL,
  "gross_margin_percentage" NUMERIC(11, 9) NOT NULL,
  "gross_income" NUMERIC(7, 4) NOT NULL,
  "Rating" NUMERIC(4, 1) NOT NULL,
  FOREIGN KEY (Product_ID) REFERENCES product(Product_ID),
  FOREIGN KEY (Customer_ID) REFERENCES customer(Customer_ID),
  FOREIGN KEY (Name) REFERENCES customer(Name),
  FOREIGN KEY (Address) REFERENCES customer(Address))''')

In [ ]:
c.executemany("INSERT INTO purchase(Invoice_ID,Product_ID,Customer_ID,Name,Address,Quantity,Total,Date,Time,Payment,cogs,gross_margin_percentage,gross_income,Rating) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?)",purchase_array)

##Table Store

In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS store(
  "Branch"	VARCHAR(1) NOT NULL PRIMARY KEY,
  "City"	VARCHAR(9) NOT NULL)''')

In [ ]:
c.executemany("INSERT OR IGNORE INTO store(Branch, City) VALUES(?,?)",store_array)

In [ ]:
MyDB.commit()

In [ ]:
MyDB.close()

In [ ]:
df = pd.read_csv(r"C:\Users\Bravy\Downloads\supermarket_sales - Sheet1.csv")
df.tail(10)

In [ ]:
df.isnull().values.any()   # Check entri NaN

In [ ]:
df.info

## Pendapatan per minggu

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
weekly_income = np.asarray(df.groupby(df.Date.dt.isocalendar().week)['gross income'].sum())

In [ ]:
weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13]
plt.title("Gross Income Tiap Minggu")
plt.xlabel("Minggu")
plt.ylabel("Gross Income")
plt.xticks(weeks)
plt.plot(weeks,weekly_income, color="#483D8B", marker='o')

## Member vs Nonmember

In [ ]:
df_normal = df[df['Customer type'] == 'Normal']
df_member = df[df['Customer type'] == 'Member']
df_normal.head()

In [ ]:
df_member

In [ ]:
avg_gross_normal = df_normal['gross income'].sum()/len(df_normal.index)
avg_gross_member = df_member['gross income'].sum()/len(df_member.index)

print(f'Average Gross Income dari customer normal: {avg_gross_normal}')
print(f'Average Gross Income dari customer member: {avg_gross_member}')

plt.title("Average Gross Income Berdasarkan Membership")
plt.xlabel("Membership")
plt.ylabel("Gross Income")
plt.bar(["Iya", "Tidak"], [avg_gross_normal, avg_gross_member], color=["#556B2F", "#8B0000"])

## Perbandingan keuntungan antar product line

In [ ]:
product_line = np.asarray(df[['Product line']])
np.unique(product_line)

In [ ]:
df_EA = df[df['Product line'] == 'Electronic accessories']
df_FA = df[df['Product line'] == 'Fashion accessories']
df_FB = df[df['Product line'] == 'Food and beverages']
df_HB = df[df['Product line'] == 'Health and beauty']
df_HL = df[df['Product line'] == 'Home and lifestyle']
df_ST = df[df['Product line'] ==  'Sports and travel']

In [ ]:
gross_EA = df_EA['gross income'].sum()
gross_FA = df_FA['gross income'].sum()
gross_FB = df_FB['gross income'].sum()
gross_HB = df_HB['gross income'].sum()
gross_HL = df_HL['gross income'].sum()
gross_ST = df_ST['gross income'].sum()

plt.title("Gross Income Tiap Lini Produk")
plt.xlabel("Nama Lini Produk")
plt.ylabel("Gross Income")
plt.bar(["Electronics", "Fashion", "Food", "Health", "Lifestyle", "Sports"], 
        [gross_EA, gross_FA, gross_FB, gross_HB, gross_HL, gross_ST], color=["#000080","#006400","#8B0000", "#483D8B", "#DC143C", "#556B2F"])

## Perbandingan keuntungan cabang

In [ ]:
cabang = np.asarray(df[['Branch']])
np.unique(cabang)

In [ ]:
df_A = df[df['Branch'] == 'A']
df_B = df[df['Branch'] == 'B']
df_C = df[df['Branch'] == 'C']

In [ ]:
gross_A = df_A['gross income'].sum()
gross_B = df_B['gross income'].sum()
gross_C = df_C['gross income'].sum()

plt.title("Gross Income per Cabang")
plt.xlabel("Nama Cabang")
plt.ylabel("Gross Income")
plt.bar(["Yangon","Naypyitaw","Mandalay"], [gross_A, gross_B, gross_C], color=["#000080","#006400","#8B0000"])

print(f'Gross Income dari cabang Yangon: {gross_A}')
print(f'Gross Income dari cabang Mandalay: {gross_B}')
print(f'Gross Income dari cabang Naypyitaw: {gross_C}')